In [1]:
import torch
import torch.nn as nn


import sys

sys.path.append('app/src')
sys.path.append('./src')
from util import sparse_tensor_stats


from sparse_evaluation_4 import SparseEvaluation  
from zono_sparse_gen import ZonoSparseGeneration

from abstract_relu import AbstractReLU
from sparse_addition_2 import SparseAddition

/home/guillaume/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 17:13:14,521	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [21]:
input = torch.randn(1,3)
input = torch.where(input>0,input,0)
print(input)
_, new_sparse = ZonoSparseGeneration(input,from_trash=True,start_index=5).total_zono()

tensor([[2.4162, 0.0000, 1.2699]])


In [22]:
new_sparse

tensor(indices=tensor([[5, 6],
                       [0, 2]]),
       values=tensor([2.4162, 1.2699]),
       size=(7, 3), nnz=2, layout=torch.sparse_coo)

In [23]:
new_sparse.to_dense()

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [2.4162, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.2699]])

In [8]:
import ray

# Simuler ray.remote pour cet exemple
def remote_decorator(num_gpus=None, num_cpus=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            resources = []
            if num_gpus is not None:
                resources.append(f"{num_gpus} GPUs")
            if num_cpus is not None:
                resources.append(f"{num_cpus} CPUs")
            print(f"Running with: {', '.join(resources)}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

class DecoratedClass:
    def __init__(self, cls, gpu_decorator, cpu_decorator):
        self.decorated_class = self._create_decorated_class(cls, gpu_decorator, cpu_decorator)

    def _create_decorated_class(self, cls, gpu_decorator, cpu_decorator):
        class Decorated(cls):
            pass
        
        for attr_name in dir(cls):
            attr = getattr(cls, attr_name)
            if callable(attr) and not attr_name.startswith("__"):
                decorated_method = gpu_decorator(attr)
                decorated_method = cpu_decorator(decorated_method)
                setattr(Decorated, attr_name, decorated_method)
                
        return Decorated

    def __call__(self, *args, **kwargs):
        return self.decorated_class(*args, **kwargs)

# Exemple de décorateurs
gpu_decorator = remote_decorator(num_gpus=1)
cpu_decorator = remote_decorator(num_cpus=4)

# Utilisation
DecoratedMyClass = DecoratedClass(MyClass, gpu_decorator, cpu_decorator)

# Instanciation et appel de la méthode décorée
instance = DecoratedMyClass()
print(instance.my_method())
print(instance)


Running with: 4 CPUs
Running with: 1 GPUs
Hello from MyClass
<__main__.DecoratedClass._create_decorated_class.<locals>.Decorated object at 0x72155eee4370>


In [13]:
ray.shutdown()

In [12]:
import ray

# Supposons que votre classe SparseWorker ressemble à ceci :
class SparseWorker:
    def __init__(self, param1, param2):
        self.param1 = param1
        self.param2 = param2

    def work(self):
        return self.param1 + self.param2

# Fonction pour instancier la classe avec des paramètres dynamiques
def create_sparse_worker(num_cpus,num_gpus, *args, **kwargs):
    SparseWorkerRemote = ray.remote(num_cpus=num_cpus,num_gpus=num_gpus)(SparseWorker)
    return SparseWorkerRemote.remote(*args, **kwargs)

# Initialiser Ray
ray.init()

# Créez une instance de SparseWorker avec un nombre personnalisé de CPUs
worker = create_sparse_worker(2,1, "param1_value", "param2_value")

# Appelez une méthode de la classe
result = ray.get(worker.work.remote())
print(result)


2024-07-10 15:12:17,051	INFO worker.py:1753 -- Started a local Ray instance.


(autoscaler +34m2s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +34m2s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


KeyboardInterrupt: 